In [23]:
from datetime import datetime
from dateutil.tz import tzlocal

import numpy as np
from pynwb import NWBFile, NWBHDF5IO, TimeSeries
from pynwb.ecephys import ElectricalSeries, LFP
from pynwb.behavior import BehavioralEvents
import pandas as pd

In [2]:
nwbfile = NWBFile(
    session_description="Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM",
    identifier="TODO",
    session_start_time=datetime(2019, 3, 27),
    lab="Johannes Sarnthein Lab",
    institution="University of Zurich",
)

/home/hhh/.cache/pypoetry/virtualenvs/usz-neuro-conversion-6s2qedde-py3.11/lib/python3.11/site-packages/pynwb/file.py:1113: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")


In [3]:
device = nwbfile.create_device(
    name="NicoletOne EEG", description="TODO", manufacturer="Natus Medical Incorporated"
)
nwbfile.add_electrode_column(name="label", description="label of electrode")

group_count = 19
channels_per_group = 1
electrode_counter = 0

for group in range(group_count):
    # create an electrode group for this group
    electrode_group = nwbfile.create_electrode_group(
        name="group{}".format(group),
        description="electrode group for group {}".format(group),
        device=device,
        location="PZ",
        position=[0, 0, 0],
    )
    # add electrodes to the electrode table
    for channel in range(channels_per_group):
        nwbfile.add_electrode(
            group=electrode_group,
            label="group{}channel{}".format(group, channel),
            location="PZ",
            reference="Averaged mastoid channels",
        )
        electrode_counter += 1
print(nwbfile.electrodes.to_dataframe())

   location                                              group group_name  \
id                                                                          
0        PZ  group0 pynwb.ecephys.ElectrodeGroup at 0x14004...     group0   
1        PZ  group1 pynwb.ecephys.ElectrodeGroup at 0x14004...     group1   
2        PZ  group2 pynwb.ecephys.ElectrodeGroup at 0x14004...     group2   
3        PZ  group3 pynwb.ecephys.ElectrodeGroup at 0x14004...     group3   
4        PZ  group4 pynwb.ecephys.ElectrodeGroup at 0x14004...     group4   
5        PZ  group5 pynwb.ecephys.ElectrodeGroup at 0x14004...     group5   
6        PZ  group6 pynwb.ecephys.ElectrodeGroup at 0x14004...     group6   
7        PZ  group7 pynwb.ecephys.ElectrodeGroup at 0x14004...     group7   
8        PZ  group8 pynwb.ecephys.ElectrodeGroup at 0x14004...     group8   
9        PZ  group9 pynwb.ecephys.ElectrodeGroup at 0x14004...     group9   
10       PZ  group10 pynwb.ecephys.ElectrodeGroup at 0x1400...    group10   

In [4]:
all_table_region = nwbfile.create_electrode_table_region(
    region=list(range(electrode_counter)),  # reference row indices 0 to N-1
    description="all electrodes",
)
# From EEG
raw_data = np.random.randn(50, electrode_counter)
raw_electrical_series = ElectricalSeries(
    name="ElectricalSeries",
    data=raw_data,
    electrodes=all_table_region,
    starting_time=0.0,
    rate=256.0,  # in Hz
)
nwbfile.add_acquisition(raw_electrical_series)
# TODO: stimuli

In [5]:
# From iEEG
lfp_data = np.random.randn(50, electrode_counter)
lfp_electrical_series = ElectricalSeries(
    name="ElectricalSeries",
    data=lfp_data,
    electrodes=all_table_region,
    starting_time=0.0,
    rate=200.0,
)
lfp = LFP(electrical_series=lfp_electrical_series)
module = nwbfile.create_processing_module(
    name="ecephys", description="processed extracellular electrophysiology data"
)
module.add(lfp)

LFP pynwb.ecephys.LFP at 0x140047392621136
Fields:
  electrical_series: {
    ElectricalSeries <class 'pynwb.ecephys.ElectricalSeries'>
  }

In [6]:
nwbfile.add_unit_column(name="quality", description="sorting quality")

poisson_lambda = 20
firing_rate = 20
n_units = 10
for n_units_per_shank in range(n_units):
    n_spikes = np.random.poisson(lam=poisson_lambda)
    spike_times = np.round(
        np.cumsum(np.random.exponential(1 / firing_rate, n_spikes)), 5
    )
    nwbfile.add_unit(
        spike_times=spike_times, quality="good", waveform_mean=[1.0, 2.0, 3.0, 4.0, 5.0]
    )

nwbfile.units.to_dataframe()

,quality,spike_times,waveform_mean
id,,,
0,good,"[0.11488, 0.12599, 0.28997, 0.36985, 0.39364, ...","[1.0, 2.0, 3.0, 4.0, 5.0]"
1,good,"[0.08816, 0.0932, 0.12707, 0.14344, 0.14377, 0...","[1.0, 2.0, 3.0, 4.0, 5.0]"
2,good,"[0.0131, 0.05536, 0.13833, 0.19851, 0.22043, 0...","[1.0, 2.0, 3.0, 4.0, 5.0]"
3,good,"[0.00671, 0.02871, 0.10425, 0.15911, 0.18597, ...","[1.0, 2.0, 3.0, 4.0, 5.0]"
4,good,"[0.08765, 0.21808, 0.33023, 0.42971, 0.44271, ...","[1.0, 2.0, 3.0, 4.0, 5.0]"
5,good,"[0.04806, 0.09087, 0.12463, 0.16793, 0.21134, ...","[1.0, 2.0, 3.0, 4.0, 5.0]"
6,good,"[0.18303, 0.21929, 0.47895, 0.48994, 0.57986, ...","[1.0, 2.0, 3.0, 4.0, 5.0]"
7,good,"[0.12654, 0.19628, 0.22617, 0.38808, 0.41525, ...","[1.0, 2.0, 3.0, 4.0, 5.0]"
8,good,"[0.03941, 0.06247, 0.06403, 0.12542, 0.23531, ...","[1.0, 2.0, 3.0, 4.0, 5.0]"


In [7]:
# TODO: Add waveforms

In [33]:
behavior_module = nwbfile.create_processing_module(
    name="Session trials data15", description="Data for all trials in this session."
)
reward_amount = np.rec.fromarrays(([1., 1.5, 1., 1.5], [1.1, 1.6, 1.1, 1.6]), names=('keys', 'data'))
events_timestamps = [1., 2., 5., 6.]

time_series = TimeSeries(
    name="lever_presses",
    data=reward_amount,
    timestamps=events_timestamps,
    description="The water amount the subject received as a reward.",
    unit="ml",
)

behavioral_events = BehavioralEvents(time_series=time_series, name="BehavioralEvents")

behavior_module.add(behavioral_events)

BehavioralEvents pynwb.behavior.BehavioralEvents at 0x140047337291728
Fields:
  time_series: {
    lever_presses <class 'pynwb.base.TimeSeries'>
  }

In [34]:
nwbfile.processing["Session trials data15"].data_interfaces["BehavioralEvents"].time_series

{'lever_presses': lever_presses pynwb.base.TimeSeries at 0x140047338493584
 Fields:
   comments: no comments
   conversion: 1.0
   data: [(1. , 1.1) (1.5, 1.6) (1. , 1.1) (1.5, 1.6)]
   description: The water amount the subject received as a reward.
   interval: 1
   offset: 0.0
   resolution: -1.0
   timestamps: [1. 2. 5. 6.]
   timestamps_unit: seconds
   unit: ml}

In [ ]:
with NWBHDF5IO("ecephys_tutorial.nwb", "w") as io:
    io.write(nwbfile)